AInewsbot.ipynb

- Automate collecting daily AI news
- Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
- Save HTML of each URL in htmldata directory
- Extract URLs from all files, create a pandas dataframe with url, title, src
- Use ChatGPT to filter only AI-related headlines by sending a prompt and formatted table of headlines
- Use SQLite to filter headlines previously seen 
- OPENAI_API_KEY should be in the environment or in a .env file
  
Alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [1]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json

import numpy as np
import pandas as pd

# import bs4
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL,
                        SOURCECONFIG, PROMPT, MAX_INPUT_TOKENS)
from ainb_utilities import log, delete_files, filter_unseen_urls_db, insert_article, nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy
from ainb_webscrape import get_driver, quit_drivers, get_file, get_url, parse_file, get_og_tags, get_path_from_url, trimmed_href, DRIVERS
from ainb_llm import paginate_df, process_pages

# needed because jupyter is already running an async event loop
import nest_asyncio
import asyncio

In [2]:
# PROMPT = """
# You will act as a research assistant to categorize news articles based on their relevance
# to the topic of artificial intelligence (AI). You will process and classify news headlines
# formatted as JSON objects.

# Input Specification:
# You will receive a list of news stories formatted as JSON objects.
# Each object will include an 'id' and a 'title'. For instance:
# [{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
#  {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
#  {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
#  {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
#  {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
#  ]

# Classification Criteria:
# Classify each story based on its title to determine whether it primarily pertains to AI.
# Broadly define AI-related content to include topics such as machine learning, robotics,
# computer vision, reinforcement learning, large language models, and related topics. Also
# include specific references to AI-related entities and individuals and products such as
# OpenAI, ChatGPT, Elon Musk, Sam Altman, Anthropic Claude, Google Gemini, Copilot,
# Perplexity.ai, Midjourney, etc.

# Output Specification:
# You will return a JSON object with the field 'stories' containing the list of classification results.
# For each story, your output will be a JSON object containing the original 'id' and a new field 'isAI',
# a boolean indicating if the story is about AI. The output schema must be strictly adhered to, without
# any additional fields. Example output:
# {'stories':
# [{'id': 97, 'isAI': true},
#  {'id': 103, 'isAI': true},
#  {'id': 103, 'isAI': false},
#  {'id': 210, 'isAI': true},
#  {'id': 298, 'isAI': false}]
# }

# Ensure that each output object accurately reflects the corresponding input object in terms of the 'id' field
# and that the 'isAI' field accurately represents the AI relevance of the story as determined by the title.

# The list of news stories to classify and enrich is:

# """

In [3]:
print(PROMPT)


You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will process and classify news headlines
formatted as JSON objects.

Input Specification:
You will receive a list of news stories formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based on its title to determine whether it primarily pertains to AI.
Broadly define AI-related content to include topics such as machine learning, robotics,
computer vision, reinforcement l

In [4]:
get_og_tags('https://druce.ai')


{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [5]:
get_path_from_url('https://druce.ai/2024/03/gemini-summarize-book')


'/2024/03/gemini-summarize-book'

In [6]:
trimmed_href('https://druce.ai/2024/03/gemini-summarize-book?xyz')


'https://druce.ai/2024/03/gemini-summarize-book'

In [7]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources")



2024-07-02 12:28:19,910 - AInewsbot - INFO - Load 17 sources


20

In [8]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW

In [9]:
# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

sources_reverse

2024-07-02 12:28:19,917 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-02 12:28:19,917 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-02 12:28:19,917 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-02 12:28:19,918 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-02 12:28:19,918 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-02 12:28:19,918 - AInewsbot - INFO - Google News -> https://news.google.com/topics/CAA

{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [10]:
# determine files in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]

# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))

2024-07-02 12:28:19,931 - AInewsbot - INFO - 17
2024-07-02 12:28:19,931 - AInewsbot - INFO - htmldata/Technology - The Washington Post (07_02_2024 10_01_29 AM).html
2024-07-02 12:28:19,935 - AInewsbot - INFO - htmldata/Technology - WSJ.com (07_02_2024 10_01_25 AM).html
2024-07-02 12:28:19,940 - AInewsbot - INFO - htmldata/Discover and Add New Feedly AI Feeds (07_02_2024 10_01_20 AM).html
2024-07-02 12:28:19,941 - AInewsbot - INFO - htmldata/AI News _ VentureBeat (07_02_2024 10_01_19 AM).html
2024-07-02 12:28:19,941 - AInewsbot - INFO - htmldata/Artificial Intelligence - The Verge (07_02_2024 10_01_14 AM).html
2024-07-02 12:28:19,941 - AInewsbot - INFO - htmldata/top scoring links _ multi (07_02_2024 10_01_08 AM).html
2024-07-02 12:28:19,942 - AInewsbot - INFO - htmldata/The Register_ Enterprise Technology News and Analysis (07_02_2024 10_01_04 AM).html
2024-07-02 12:28:19,942 - AInewsbot - INFO - htmldata/Techmeme (07_02_2024 10_00_53 AM).html
2024-07-02 12:28:19,942 - AInewsbot - INFO

In [12]:
# Fetch HTML files from sources

# takes 5 minutes without multiprocessing
# get driver takes 50 seconds, so call it 1 + 4
# 2 drivers should take 2 + 2, 3: 3 + 4/3
# could save a minute by using 2 webdrivers

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
log("Saving HTML files")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and fetchuntil none are left
# could probably just access queue as a global or just define 1 function
# this pattern lets you create an array of functions with different args

num_browsers = 4
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args

# what you need to fix here - queue should be id, url, title
# # map id back to sourcedict
# def process_queue_factory(q):
#     def process_queue():
#         # launch browser via selenium driver
#         driver = webscrape.get_driver()    
#         saved_pages = []
#         while not q.empty():
#             i, url, title = q.get()
#             webscrape.log(f'Processing {url}')
#             savefile = webscrape.get_url(url, driver)
#             saved_pages.append((i, url, title, savefile))
#         # Close the browser
#         webscrape.log("Quit webdriver")
#         driver.quit()
#         return saved_pages
#     return process_queue

# callable = process_queue_factory(queue)
# # saved_pages = callable()

# def launch_drivers(num_drivers):
#     with ThreadPoolExecutor(max_workers=num_drivers) as executor:
#         # Create a list of future objects
#         futures = [executor.submit(callable) for _ in range(num_drivers)]
        
#         # Collect the results (web drivers) as they complete
#         retarray = [future.result() for future in as_completed(futures)]
        
#     return retarray

# print(f"fetching {len(df)} pages using {num_browsers} browsers")
# results = launch_drivers(num_browsers)
# ---
def process_queue_factory(q):
    def process_queue():
        # launch browser via selenium driver
        driver = get_driver()    
        saved_pages = []
        while not q.empty():
            sourcedict = q.get()
            sourcename = sourcedict['sourcename']
            log(f'Processing {sourcename}')
            sourcefile = get_file(sourcedict, driver)
            saved_pages.append((sourcename, sourcefile))  
        # Close the browser
        log("Quit webdriver")
        driver.quit()
        return saved_pages
    return process_queue

callable = process_queue_factory(queue)
# saved_pages = callable()

def launch_drivers(num_drivers):
    with ThreadPoolExecutor(max_workers=num_drivers) as executor:
        # Create a list of future objects
        futures = [executor.submit(callable) for _ in range(num_drivers)]
        
        # Collect the results (web drivers) as they complete
        retarray = [future.result() for future in as_completed(futures)]
        
    return retarray

results = launch_drivers(3)

2024-07-02 12:37:23,613 - AInewsbot - INFO - Saving HTML files
2024-07-02 12:37:23,640 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver
2024-07-02 12:37:23,640 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver
2024-07-02 12:37:23,640 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver
2024-07-02 12:37:41,008 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:37:41,008 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:37:41,009 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:37:41,009 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:37:41,009 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:37:41,009 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:38:29,377 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-02 12:38:29,374 - AInewsbot - INFO - get_driver - Initialize

2024-07-02 12:39:15,156 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (07_02_2024 12_39_15 PM).html as utf-8
2024-07-02 12:39:15,174 - AInewsbot - INFO - Processing NYT Tech
2024-07-02 12:39:15,174 - AInewsbot - INFO - get_files(Technology - The New York Times) - starting get_files https://www.nytimes.com/section/technology
2024-07-02 12:39:21,900 - AInewsbot - INFO - get_files(Discover and Add New Feedly AI Feeds) - Loading additional infinite scroll items
2024-07-02 12:39:23,322 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrom

2024-07-02 12:40:08,506 - AInewsbot - INFO - get_files(Technology - WSJ.com) - Saving Technology - WSJ.com (07_02_2024 12_40_08 PM).html as utf-8
2024-07-02 12:40:08,507 - AInewsbot - INFO - Quit webdriver
2024-07-02 12:40:13,386 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-02 12:40:13,386 - AInewsbot - INFO - get_files(Technology - The Washington Post) - Saving Technology - The Washington Post (07_02_2024 12_40_13 PM).html as utf-8
2024-07-02 12:40:13,388 - AInewsbot - INFO - Quit webdriver


In [13]:
# flatten results
saved_pages = []
for r in results:
    saved_pages.extend(r)
saved_pages

[('Ars Technica', 'htmldata/Ars Technica (07_02_2024 12_38_41 PM).html'),
 ('FT Tech', 'htmldata/Technology (07_02_2024 12_38_51 PM).html'),
 ('Hacker News', 'htmldata/Hacker News Page 1 (07_02_2024 12_39_02 PM).html'),
 ('Hacker News 2',
  'htmldata/Hacker News Page 2 (07_02_2024 12_39_12 PM).html'),
 ('HackerNoon',
  'htmldata/HackerNoon - read, write and learn about any technology (07_02_2024 12_39_23 PM).html'),
 ('Reddit',
  'htmldata/top scoring links _ multi (07_02_2024 12_39_56 PM).html'),
 ('VentureBeat',
  'htmldata/AI News _ VentureBeat (07_02_2024 12_40_06 PM).html'),
 ('Business Insider',
  'htmldata/Tech - Business Insider (07_02_2024 12_38_43 PM).html'),
 ('Google News',
  'htmldata/Google News - Technology - Artificial intelligence (07_02_2024 12_39_15 PM).html'),
 ('NYT Tech',
  'htmldata/Technology - The New York Times (07_02_2024 12_39_25 PM).html'),
 ('Techmeme', 'htmldata/Techmeme (07_02_2024 12_39_36 PM).html'),
 ('The Register',
  'htmldata/The Register_ Enterpri

In [14]:
# gives error, TypeError: cannot pickle '_thread.lock' object
# not sure why, should be straightforward
# overkill to save a minute but 2 ways to do it:
#  function that takes the queue as an arg, gets driver, keeps popping items until queue is empty, quits driver 
#  use a pool.map() on a function that takes a single url as an arg, after all complete, call quit_drivers()

# import multiprocessing
# from queue import Queue

# # Function to download and save the web page
# def download_page(url_queue, download_dir):
#     driver = get_driver()

#     while not url_queue.empty():
#         url = url_queue.get()
#         log(url)
#         try:
#             driver.get(url)
#             # Save page source
#             file_name = os.path.join(download_dir, f"{url_queue.qsize()}.html")
#             with open(file_name, 'w', encoding='utf-8') as file:
#                 file.write(driver.page_source)
#             print(f"Saved {url} to {file_name}")
#         except Exception as e:
#             print(f"Failed to download {url}: {e}")
#         finally:
#             url_queue.task_done()

#     driver.quit()

# # Create a queue and add URLs
# url_queue = Queue()
# urls = [v['url'] for v in sources.values()]
# for url in urls:
#     url_queue.put(url)

# # Create and start threads
# processes = []

# for i in range(3):  # 3 Selenium WebDriver instances
#     process = multiprocessing.Process(target=download_page, args=(url_queue, DOWNLOAD_DIR))
#     process.start()
#     processes.append(thread)

# # Wait for all threads to finish
# for process in processes:
#     process.join()

# print("All pages have been downloaded.")

In [15]:
print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    print(sourcename, '->', page)
    

17
Ars Technica -> htmldata/Ars Technica (07_02_2024 12_38_41 PM).html
FT Tech -> htmldata/Technology (07_02_2024 12_38_51 PM).html
Hacker News -> htmldata/Hacker News Page 1 (07_02_2024 12_39_02 PM).html
Hacker News 2 -> htmldata/Hacker News Page 2 (07_02_2024 12_39_12 PM).html
HackerNoon -> htmldata/HackerNoon - read, write and learn about any technology (07_02_2024 12_39_23 PM).html
Reddit -> htmldata/top scoring links _ multi (07_02_2024 12_39_56 PM).html
VentureBeat -> htmldata/AI News _ VentureBeat (07_02_2024 12_40_06 PM).html
Business Insider -> htmldata/Tech - Business Insider (07_02_2024 12_38_43 PM).html
Google News -> htmldata/Google News - Technology - Artificial intelligence (07_02_2024 12_39_15 PM).html
NYT Tech -> htmldata/Technology - The New York Times (07_02_2024 12_39_25 PM).html
Techmeme -> htmldata/Techmeme (07_02_2024 12_39_36 PM).html
The Register -> htmldata/The Register_ Enterprise Technology News and Analysis (07_02_2024 12_39_47 PM).html
The Verge -> htmldat

In [16]:
# for sourcename, filename in saved_pages:
#     sources[sourcename]["latest"]=filename
#     sources[sourcename]["sourcename"]=sourcename
    
    

In [17]:
# Parse news URLs and titles from downloaded HTML files
log("parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    print(sourcename, '->', filename, flush=True)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

2024-07-02 12:40:15,771 - AInewsbot - INFO - parsing html files


Ars Technica -> htmldata/Ars Technica (07_02_2024 12_38_41 PM).html


2024-07-02 12:40:15,773 - AInewsbot - INFO - parse loop - Ars Technica
2024-07-02 12:40:15,814 - AInewsbot - INFO - parse_file - found 252 raw links
2024-07-02 12:40:15,819 - AInewsbot - INFO - parse_file - found 31 filtered links
2024-07-02 12:40:15,820 - AInewsbot - INFO - parse loop - 31 links found


FT Tech -> htmldata/Technology (07_02_2024 12_38_51 PM).html


2024-07-02 12:40:15,820 - AInewsbot - INFO - parse loop - FT Tech
2024-07-02 12:40:15,850 - AInewsbot - INFO - parse_file - found 460 raw links
2024-07-02 12:40:15,855 - AInewsbot - INFO - parse_file - found 105 filtered links
2024-07-02 12:40:15,855 - AInewsbot - INFO - parse loop - 105 links found


...

Feedly AI -> htmldata/Discover and Add New Feedly AI Feeds (07_02_2024 12_40_01 PM).html


2024-07-02 12:40:16,684 - AInewsbot - INFO - parse loop - Feedly AI
2024-07-02 12:40:16,711 - AInewsbot - INFO - parse_file - found 249 raw links
2024-07-02 12:40:16,714 - AInewsbot - INFO - parse_file - found 65 filtered links
2024-07-02 12:40:16,715 - AInewsbot - INFO - parse loop - 65 links found


WaPo Tech -> htmldata/Technology - The Washington Post (07_02_2024 12_40_13 PM).html


2024-07-02 12:40:16,715 - AInewsbot - INFO - parse loop - WaPo Tech
2024-07-02 12:40:16,728 - AInewsbot - INFO - parse_file - found 156 raw links
2024-07-02 12:40:16,730 - AInewsbot - INFO - parse_file - found 25 filtered links
2024-07-02 12:40:16,731 - AInewsbot - INFO - parse loop - 25 links found
2024-07-02 12:40:16,732 - AInewsbot - INFO - parse loop - found 1599 links


20

In [18]:
# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
print(len(orig_df))
orig_df.head()

1300


,id,src,title,url
0,0,Ars Technica,Alzheimer’s scientist indicted for allegedly f...,https://arstechnica.com/science/2024/07/alzhei...
1,1,Ars Technica,Surface Pro 11 and Laptop 7 review: An Apple S...,https://arstechnica.com/gadgets/2024/07/surfac...
2,2,Ars Technica,The newRivenremake is even better thanMyst,https://arstechnica.com/gaming/2024/06/the-new...
3,3,Ars Technica,Paul Sutter walks us through the future of cli...,https://arstechnica.com/science/2022/04/paul-s...
4,4,Ars Technica,An ultra-athlete goes head-to-head with the wo...,https://arstechnica.com/science/2024/06/an-ult...


In [19]:
# datestr = '2024-06-20'

# conn = sqlite3.connect('articles.db')

# c = conn.cursor()
# query = f"select * from news_articles where article_date > '{datestr}' order by id desc limit 244"
# df = pd.read_sql_query(query, conn)
# filtered_df=df.reset_index(). \
#   drop(columns='id'). \
#   rename(columns={'index': 'id'})

# filtered_df



In [20]:
# conn.execute(f"delete from news_articles where article_date > '{datestr}'")

# # Committing the changes
# conn.commit()

# # Closing the connection
# conn.close()


In [21]:
filtered_df = filter_unseen_urls_db(orig_df)


2024-07-02 12:40:17,085 - AInewsbot - INFO - Existing URLs: 119065
2024-07-02 12:40:17,106 - AInewsbot - INFO - New URLs: 291


In [22]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI

client = OpenAI()

# make pages that fit in a reasonably sized prompt
pages = paginate_df(filtered_df)

In [23]:
pages

[[{'id': 1,
   'title': 'Surface Pro 11 and Laptop 7 review: An Apple Silicon moment for Windows'},
  {'id': 19,
   'title': 'Call the ant doctor: Amputation gives injured ants a leg up on infections'},
  {'id': 28,
   'title': 'Tesla posts disappointing production and sales numbers for Q2 2024'},
  {'id': 30,
   'title': '“People, planet, profit”—how Michelin approaches sustainability'},
  {'id': 66,
   'title': 'Northern Data May Be Testing Limits of AI Frenzy With Unit IPO'},
  {'id': 95,
   'title': 'YouTube is really good at getting kids to buy stuff and way ahead of its competition, new data shows'},
  {'id': 96,
   'title': 'Forget TikTok, influencers say Instagram reels and YouTube shorts are better for creators getting started to grow an audience'},
  {'id': 110,
   'title': "This company sends rejection feedback emails to every job applicant. It's made a big difference."},
  {'id': 127,
   'title': 'Marine Le Pen says she will seek to form government even if short of outright

In [24]:
# updated ainb_llm.py to use async wait
# the lowest level  client.chat.completions.create call does not support await
# gives object ChatCompletion can't be used in 'await' expression
# this still takes 2 minutes for 456 URLs
# the OpenAI python API still blocks / throttles

# print(datetime.now())
# enriched_urls = await process_pages(client, PROMPT, pages)
# print(datetime.now())


In [25]:
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}



In [26]:
# this runs fast with async aiohttp (15 seconds vs 2 minutes)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()

async def fetch_openai(session, payload):
    async with session.post(API_URL, headers=headers, json=payload) as response:
        return await response.json()


async def fetch_pages(prompt, pages):
    log(f"{datetime.now().strftime('%H:%M:%S')} Sending {len(pages)} pages to OpenAI.")

    # make a prompt and payload for each page
    payloads = [{"model":  LOWCOST_MODEL,
                 "response_format": {"type": "json_object"},
                 "messages": [{"role": "user",
                               "content": prompt + json.dumps(p)
                               }]
                 } for p in pages]

    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, payload in enumerate(payloads):
            log(f"{datetime.now().strftime('%H:%M:%S')} Sending page {i}.")
            task = asyncio.create_task(fetch_openai(session, payload))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)

    # validate and process the responses
    log(f"{datetime.now().strftime('%H:%M:%S')} Processing responses... ")

    retlist = []
    for i, response in enumerate(responses):
        try:
            response_dict = json.loads(
                response["choices"][0]["message"]["content"])
        except Exception as e:
            raise TypeError("Error: Invalid response " + str(e))

        if type(response_dict) is dict:
            response_list = response_dict.get("stories")
        else:
            raise TypeError("Error: Invalid response type")

        if type(response_list) is not list:
            raise TypeError("Error: Invalid response type")

        log(f"{datetime.now().strftime('%H:%M:%S')} got list with {len(response_list)} items ")

        # check all sent present in response
        # seems to match, i.e. responses received correctly and in order
        # could add this to fetch_openai and implement retry
        sent_ids = [s['id'] for s in pages[i]]
        received_ids = [r['id'] for r in response_list]
        difference = set(sent_ids) - set(received_ids)

        if difference:
            log(f"missing items, {str(difference)}")

        retlist.extend(response_list)

    log(f"{datetime.now().strftime('%H:%M:%S')} Processed {len(retlist)} responses.")

    return retlist


# Run the main function
print(datetime.now())
enriched_urls = asyncio.run(fetch_pages(PROMPT, pages))
print(datetime.now())


2024-07-02 12:40:17,440 - AInewsbot - INFO - 12:40:17 Sending 6 pages to OpenAI.
2024-07-02 12:40:17,441 - AInewsbot - INFO - 12:40:17 Sending page 0.
2024-07-02 12:40:17,441 - AInewsbot - INFO - 12:40:17 Sending page 1.
2024-07-02 12:40:17,441 - AInewsbot - INFO - 12:40:17 Sending page 2.
2024-07-02 12:40:17,441 - AInewsbot - INFO - 12:40:17 Sending page 3.
2024-07-02 12:40:17,441 - AInewsbot - INFO - 12:40:17 Sending page 4.
2024-07-02 12:40:17,441 - AInewsbot - INFO - 12:40:17 Sending page 5.


2024-07-02 12:40:17.439324


2024-07-02 12:40:30,256 - AInewsbot - INFO - 12:40:30 Processing responses... 
2024-07-02 12:40:30,257 - AInewsbot - INFO - 12:40:30 got list with 50 items 
2024-07-02 12:40:30,258 - AInewsbot - INFO - 12:40:30 got list with 50 items 
2024-07-02 12:40:30,259 - AInewsbot - INFO - 12:40:30 got list with 50 items 
2024-07-02 12:40:30,261 - AInewsbot - INFO - 12:40:30 got list with 50 items 
2024-07-02 12:40:30,261 - AInewsbot - INFO - 12:40:30 got list with 50 items 
2024-07-02 12:40:30,262 - AInewsbot - INFO - 12:40:30 got list with 41 items 
2024-07-02 12:40:30,263 - AInewsbot - INFO - 12:40:30 Processed 291 responses.


2024-07-02 12:40:30.265365


In [27]:
enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
enriched_df.head()


291


,id,isAI
0,1,False
1,19,False
2,28,False
3,30,False
4,66,True


In [28]:
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))


2024-07-02 12:40:30,281 - AInewsbot - INFO - 113 - isAI
2024-07-02 12:40:30,282 - AInewsbot - INFO - 178 - not isAI


20

In [29]:
# merge returned df into original df
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,1,Ars Technica,Surface Pro 11 and Laptop 7 review: An Apple S...,https://arstechnica.com/gadgets/2024/07/surfac...,False,2024-07-02
1,19,Ars Technica,Call the ant doctor: Amputation gives injured ...,https://arstechnica.com/science/2024/07/call-t...,False,2024-07-02
2,28,Ars Technica,Tesla posts disappointing production and sales...,https://arstechnica.com/cars/2024/07/tesla-pro...,False,2024-07-02
3,30,Ars Technica,"“People, planet, profit”—how Michelin approach...",https://arstechnica.com/cars/2024/07/heres-how...,False,2024-07-02
4,66,Bloomberg Tech,Northern Data May Be Testing Limits of AI Fren...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-02


In [30]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-02 12:40:30,296 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-02 12:40:30,297 - AInewsbot - INFO - Unmatched source rows: 0


20

In [31]:
# update SQLite database with all seen articles
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()
for row in merged_df.itertuples():
    insert_article(conn, cursor, row.src, row.title,
                   row.url, row.isAI, row.date)


In [32]:
AIdf = merged_df.loc[merged_df["isAI"]==1].reset_index(drop=True)
log(f"Found {len(AIdf)} AI headlines")


2024-07-02 12:40:30,374 - AInewsbot - INFO - Found 113 AI headlines


20

In [33]:
AIdf = merged_df.loc[merged_df["isAI"]].reset_index(drop=True)
log(f"Found {len(AIdf)} AI headlines")


2024-07-02 12:40:30,378 - AInewsbot - INFO - Found 113 AI headlines


20

In [34]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

def unicode_to_ascii(input_string):
    # Normalize the Unicode string to NFKD form
    normalized_string = unicodedata.normalize('NFKD', input_string)
    
    # Encode to ASCII bytes, ignoring characters that cannot be converted
    ascii_bytes = normalized_string.encode('ascii', 'ignore')
    
    # Convert bytes back to a string
    ascii_string = ascii_bytes.decode('ascii')
    
    return ascii_string

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [35]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index()
log(f"Found {len(AIdf)} unique AI headlines")


2024-07-02 12:40:30,405 - AInewsbot - INFO - Found 111 unique AI headlines


20

In [36]:
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf = AIdf.iloc[sorted_indices].reset_index(drop=True)
AIdf


2024-07-02 12:40:30,408 - AInewsbot - INFO - Fetching embeddings for 111 headlines
2024-07-02 12:40:30,945 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


,title_clean,id,src,title,url,isAI,date
0,AI-CryptoStartupRaises$85MillioninDealLedbyThi...,438,Google News,AI-Crypto Startup Raises $85 Million in Deal L...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-02
1,Sentientcloses$85Mseedroundforopen-sourceAI,508,Google News,Sentient closes $85M seed round for open-sourc...,https://news.google.com/articles/CBMiR2h0dHBzO...,True,2024-07-02
2,PeterThiel'sFoundersFundLeads$85MSeedInvestmen...,353,Google News,Peter Thiel's Founders Fund Leads $85M Seed In...,https://news.google.com/articles/CBMiiAFodHRwc...,True,2024-07-02
3,PeterThiel'sFoundersFundco-leads$85millionseed...,624,Google News,Peter Thiel's Founders Fund co-leads $85 milli...,https://news.google.com/articles/CBMiWWh0dHBzO...,True,2024-07-02
4,"ElonMusksaysGrok2dueinAugust,Grok3byendofyear",639,Google News,"Elon Musk says Grok 2 due in August, Grok 3 by...",https://news.google.com/articles/CBMiVmh0dHBzO...,True,2024-07-02
...,...,...,...,...,...,...,...
106,"TWISTEDPICTexasman,30,arrestedafterusingdeepfa...",208,Feedly AI,"TWISTED PIC Texas man, 30, arrested after usin...",https://www.the-sun.com/news/11788500/roman-sh...,True,2024-07-02
107,PowerWashSimulatoraddstenXboxachievementswithA...,641,Google News,PowerWash Simulator adds ten Xbox achievements...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-02
108,NewsoutletsareaccusingPerplexityofplagiarisman...,196,Feedly AI,News outlets are accusing Perplexity of plagia...,https://techcrunch.com/2024/07/02/news-outlets...,True,2024-07-02
109,Assassin'sCreed'fans'areattemptingtogetShadows...,572,Google News,Assassin's Creed 'fans' are attempting to get ...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-02


In [37]:
# leaf_order = agglomerative_cluster_sort(embedding_df)
# AIdf = AIdf.iloc[leaf_order]


In [38]:
AIdf=AIdf.reset_index(drop=True)
with pd.option_context('display.max_rows', None):
    display(AIdf[["title"]])

,title
0,AI-Crypto Startup Raises $85 Million in Deal L...
1,Sentient closes $85M seed round for open-sourc...
2,Peter Thiel's Founders Fund Leads $85M Seed In...
3,Peter Thiel's Founders Fund co-leads $85 milli...
4,"Elon Musk says Grok 2 due in August, Grok 3 by..."
5,Elon Musk Announces JARVIS-Inspired xAI Grok 2...
6,Elon Musk Reveals The Staggering Amount Of Nvi...
7,"Elon Musks xAI Using 100,000 NVIDIA H100 GPUs ..."
8,Europes Bitcoin Mining Giant Northern Data Eye...
9,Northern Data May Be Testing Limits of AI Fren...


In [39]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.src}]({row.url})")
    html_str += f'{row.Index}.<a href="{row.url}">{row.title} - {row.src}</a><br />\n'


2024-07-02 12:40:31,395 - AInewsbot - INFO - [0. AI-Crypto Startup Raises $85 Million in Deal Led by Thiel's Fund - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9haS1jcnlwdG8tc3RhcnR1cC1yYWlzZXMtODUtMTMwMDAwNTUzLmh0bWzSAQA)
2024-07-02 12:40:31,395 - AInewsbot - INFO - [1. Sentient closes $85M seed round for open-source AI - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vY29pbnRlbGVncmFwaC5jb20vbmV3cy9zZW50aWVudC04NS1taWxsaW9uLXJvdW5kLW9wZW4tc291cmNlLWFp0gEA)
2024-07-02 12:40:31,395 - AInewsbot - INFO - [2. Peter Thiel's Founders Fund Leads $85M Seed Investment Into Open-Source AI Platform Sentient - Google News](https://news.google.com/articles/CBMiiAFodHRwczovL3d3dy5jb2luZGVzay5jb20vYnVzaW5lc3MvMjAyNC8wNy8wMi9wZXRlci10aGllbHMtZm91bmRlcnMtZnVuZC1sZWFkcy04NW0tc2VlZC1pbnZlc3RtZW50LWludG8tb3Blbi1zb3VyY2UtYWktcGxhdGZvcm0tc2VudGllbnQv0gGMAWh0dHBzOi8vd3d3LmNvaW5kZXNrLmNvbS9idXNpbmVzcy8yMDI0LzA3LzAyL3BldGVyLXRoaWVscy1mb3VuZGVycy1md

2024-07-02 12:40:31,398 - AInewsbot - INFO - [24. AI Firm ElevenLabs Sets Audio Reader Pact With Judy Garland, James Dean, Burt Reynolds and Laurence Olivier Estates - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vdmFyaWV0eS5jb20vMjAyNC9iaXovbmV3cy9haS1qdWR5LWdhcmxhbmQtamFtZXMtZGVhbi1idXJ0LXJleW5vbGRzLWVsZXZlbmxhYnMtcmVhZGVyLWFwcC0xMjM2MDU4ODI5L9IBcGh0dHBzOi8vdmFyaWV0eS5jb20vMjAyNC9iaXovbmV3cy9haS1qdWR5LWdhcmxhbmQtamFtZXMtZGVhbi1idXJ0LXJleW5vbGRzLWVsZXZlbmxhYnMtcmVhZGVyLWFwcC0xMjM2MDU4ODI5L2FtcC8)
2024-07-02 12:40:31,398 - AInewsbot - INFO - [25. AI goes to Hollywood: James Dean, Judy Garland and other stars will read to you now - for free - Google News](https://news.google.com/articles/CBMicGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2FpLWdvZXMtdG8taG9sbHl3b29kLWphbWVzLWRlYW4tanVkeS1nYXJsYW5kLWFuZC1vdGhlci1zdGFycy13aWxsLXJlYWQtdG8teW91LW5vdy_SAQA)
2024-07-02 12:40:31,398 - AInewsbot - INFO - [26. Geoffrey Rush on AI, Why Animation Is His Game Show Specialty and His New Fil

2024-07-02 12:40:31,401 - AInewsbot - INFO - [50. While some white collar workers may be worried, logistics and transport sectors are optimistic on AI - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vZGlnaW5vbWljYS5jb20vd2hpdGUtY29sbGFyLXdvcmtlcnMtd29ycmllZC1sb2dpc3RpY3MtdHJhbnNwb3J0LXNlY3RvcnMtb3B0aW1pc3RpYy1hadIBYWh0dHBzOi8vZGlnaW5vbWljYS5jb20vd2hpdGUtY29sbGFyLXdvcmtlcnMtd29ycmllZC1sb2dpc3RpY3MtdHJhbnNwb3J0LXNlY3RvcnMtb3B0aW1pc3RpYy1haT9hbXA)
2024-07-02 12:40:31,401 - AInewsbot - INFO - [51. Robots, AI, EW: How War in Ukraine Re-Shapes Military Tech - HackerNoon](https://hackernoon.com/robots-ai-ew-how-war-in-ukraine-re-shapes-military-tech)
2024-07-02 12:40:31,401 - AInewsbot - INFO - [52. New satellite aims to show how AI advances Earth observation - Feedly AI](https://phys.org/news/2024-07-satellite-aims-ai-advances-earth.html)
2024-07-02 12:40:31,401 - AInewsbot - INFO - [53. Running AI in the Field Could Boost Farming: Study - Google News](https://news.google.com/

2024-07-02 12:40:31,405 - AInewsbot - INFO - [79. Nvidia Faces First AI Antitrust Charge, According to Report - Google News](https://news.google.com/articles/CBMiPWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL252aWRpYS1hbnRpdHJ1c3QtYWktNzhjMWM1NDjSAUFodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvbnZpZGlhLWFudGl0cnVzdC1haS03OGMxYzU0OA)
2024-07-02 12:40:31,405 - AInewsbot - INFO - [80. Galaxy S25 to offer AI upscaling for games thanks to its SoC - Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vd3d3LmFuZHJvaWRoZWFkbGluZXMuY29tLzIwMjQvMDcvZ2FsYXh5LXMyNS10by1vZmZlci1haS11cHNjYWxpbmctZm9yLWdhbWVzLmh0bWzSAQA)
2024-07-02 12:40:31,405 - AInewsbot - INFO - [81. Snapdragon 8 Gen 4: Adreno 830 GPU rumoured to launch with DLSS Frame Generation-like feature - Google News](https://news.google.com/articles/CBMiiAFodHRwczovL3d3dy5ub3RlYm9va2NoZWNrLm5ldC9TbmFwZHJhZ29uLTgtR2VuLTQtQWRyZW5vLTgzMC1HUFUtcnVtb3VyZWQtdG8tbGF1bmNoLXdpdGgtRExTUy1GcmFtZS1HZW5lcmF0aW9uLWxpa2UtZmVhdHVyZS44NTQwMDAuMC

2024-07-02 12:40:31,408 - AInewsbot - INFO - [107. PowerWash Simulator adds ten Xbox achievements with Alice's Adventures DLC - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRydWVhY2hpZXZlbWVudHMuY29tL25ld3MvYWxpY2VzLWFkdmVudHVyZXMtc3BlY2lhbC1wYWNrLWRsYy1hY2hpZXZlbWVudHPSAQA)
2024-07-02 12:40:31,409 - AInewsbot - INFO - [108. News outlets are accusing Perplexity of plagiarism and unethical web scraping - Feedly AI](https://techcrunch.com/2024/07/02/news-outlets-are-accusing-perplexity-of-plagiarism-and-unethical-web-scraping/)
2024-07-02 12:40:31,409 - AInewsbot - INFO - [109. Assassin's Creed 'fans' are attempting to get Shadows cancelled - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmdhbWluZ2JpYmxlLmNvbS9uZXdzL2Fzc2Fzc2lucy1jcmVlZC1zaGFkb3dzLWF0dGVtcHRpbmctY2FuY2VsbGVkLTE0NTU5Ny0yMDI0MDcwMtIBAA)
2024-07-02 12:40:31,409 - AInewsbot - INFO - [110. Call of Dutys New Extra Crispy Gun Lets You Cook, Eat, and Turn Your Enemies Into Fried Chicken

In [40]:
log("Sending mail")
from_addr = os.getenv("GMAIL_USER")
to_addr = os.getenv("GMAIL_USER")
subject = 'AI news ' + datetime.now().strftime('%H:%M:%S')
body = f"""
<html>
    <head></head>
    <body>
    <div>
    {html_str}
    </div>
    </body>
</html>
"""

# Setup the MIME
message = MIMEMultipart()
message['From'] = os.getenv("GMAIL_USER")
message['To'] = os.getenv("GMAIL_USER")
message['Subject'] = subject
message.attach(MIMEText(body, 'html'))

# Create SMTP session
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()  # Secure the connection
    server.login(os.getenv("GMAIL_USER"), os.getenv("GMAIL_PASSWORD"))
    text = message.as_string()
    server.sendmail(from_addr, to_addr, text)

log("Finished")


2024-07-02 12:40:31,412 - AInewsbot - INFO - Sending mail
2024-07-02 12:40:32,698 - AInewsbot - INFO - Finished


20

In [41]:
AIdf


,title_clean,id,src,title,url,isAI,date
0,AI-CryptoStartupRaises$85MillioninDealLedbyThi...,438,Google News,AI-Crypto Startup Raises $85 Million in Deal L...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-02
1,Sentientcloses$85Mseedroundforopen-sourceAI,508,Google News,Sentient closes $85M seed round for open-sourc...,https://news.google.com/articles/CBMiR2h0dHBzO...,True,2024-07-02
2,PeterThiel'sFoundersFundLeads$85MSeedInvestmen...,353,Google News,Peter Thiel's Founders Fund Leads $85M Seed In...,https://news.google.com/articles/CBMiiAFodHRwc...,True,2024-07-02
3,PeterThiel'sFoundersFundco-leads$85millionseed...,624,Google News,Peter Thiel's Founders Fund co-leads $85 milli...,https://news.google.com/articles/CBMiWWh0dHBzO...,True,2024-07-02
4,"ElonMusksaysGrok2dueinAugust,Grok3byendofyear",639,Google News,"Elon Musk says Grok 2 due in August, Grok 3 by...",https://news.google.com/articles/CBMiVmh0dHBzO...,True,2024-07-02
...,...,...,...,...,...,...,...
106,"TWISTEDPICTexasman,30,arrestedafterusingdeepfa...",208,Feedly AI,"TWISTED PIC Texas man, 30, arrested after usin...",https://www.the-sun.com/news/11788500/roman-sh...,True,2024-07-02
107,PowerWashSimulatoraddstenXboxachievementswithA...,641,Google News,PowerWash Simulator adds ten Xbox achievements...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-02
108,NewsoutletsareaccusingPerplexityofplagiarisman...,196,Feedly AI,News outlets are accusing Perplexity of plagia...,https://techcrunch.com/2024/07/02/news-outlets...,True,2024-07-02
109,Assassin'sCreed'fans'areattemptingtogetShadows...,572,Google News,Assassin's Creed 'fans' are attempting to get ...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-02


In [42]:
# fetch pages
# Create a queue for multiprocessing and populate it 
print("Enqueuing URLs")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.url, row.title))
    

Enqueuing URLs


In [43]:
get_url


<function ainb_webscrape.get_url(url, driver=None)>

In [44]:
# process queue asynchronously
num_browsers = 4
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
def process_queue_factory(q):
    def process_queue():
        # launch browser via selenium driver
        driver = get_driver()    
        saved_pages = []
        while not q.empty():
            i, url, title = q.get()
            log(f'Processing {url}')
            savefile = get_url(url, driver)
            saved_pages.append((i, url, title, savefile))
        # Close the browser
        log("Quit webdriver")
        driver.quit()
        return saved_pages
    return process_queue

callable = process_queue_factory(queue)
# saved_pages = callable()

def launch_drivers(num_drivers):
    with ThreadPoolExecutor(max_workers=num_drivers) as executor:
        # Create a list of future objects
        futures = [executor.submit(callable) for _ in range(num_drivers)]
        
        # Collect the results (web drivers) as they complete
        retarray = [future.result() for future in as_completed(futures)]
        
    return retarray

print(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
results = launch_drivers(num_browsers)


2024-07-02 12:40:32,732 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver
2024-07-02 12:40:32,733 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver
2024-07-02 12:40:32,733 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver
2024-07-02 12:40:32,733 - AInewsbot - INFO - get_driver - 22257 Initializing webdriver


fetching 111 pages using 4 browsers


2024-07-02 12:40:52,134 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:40:52,135 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:40:52,135 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:40:52,135 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-02 12:40:52,135 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:40:52,136 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:40:52,136 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:40:52,137 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-02 12:41:51,276 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-02 12:41:51,363 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiSWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9haS1jcnlwdG8tc3RhcnR1cC1yYWlzZXMtODUtMTMwMDAwNTUzLmh0bWzSAQA
2024-07-02 12:41:51

2024-07-02 12:42:07,783 - AInewsbot - INFO - get_url(Sentient_closes__85M_seed_round_for_open-source_AI) - Saving Sentient_closes__85M_seed_round_for_open-source_AI_20240702_124207.html as utf-8
2024-07-02 12:42:07,786 - AInewsbot - INFO - Processing https://www.benzinga.com/news/24/07/39594885/elon-musk-reveals-the-staggering-amount-of-nvidia-h100-gpus-that-will-be-used-to-train-xai-chatbot-g
2024-07-02 12:42:07,796 - AInewsbot - INFO - get_url(https://www.benzinga.com/news/24/07/39594885/elon-musk-reveals-the-staggering-amount-of-nvidia-h100-gpus-that-will-be-used-to-train-xai-chatbot-g) - starting get_files https://www.benzinga.com/news/24/07/39594885/elon-musk-reveals-the-staggering-amount-of-nvidia-h100-gpus-that-will-be-used-to-train-xai-chatbot-g
2024-07-02 12:42:08,564 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#n

2024-07-02 12:42:18,990 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiZWh0dHBzOi8vZm9ydHVuZS5jb20vMjAyNC8wNy8wMi9rZW4tZ3JpZmZpbi1jaXRhZGVsLWdlbmVyYXRpdmUtYWktaHlwZS1vcGVuYWktbWlyYS1tdXJhdGktbnZpZGlhLWpvYnMv0gEA
2024-07-02 12:42:18,992 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiZWh0dHBzOi8vZm9ydHVuZS5jb20vMjAyNC8wNy8wMi9rZW4tZ3JpZmZpbi1jaXRhZGVsLWdlbmVyYXRpdmUtYWktaHlwZS1vcGVuYWktbWlyYS1tdXJhdGktbnZpZGlhLWpvYnMv0gEA) - starting get_files https://news.google.com/articles/CBMiZWh0dHBzOi8vZm9ydHVuZS5jb20vMjAyNC8wNy8wMi9rZW4tZ3JpZmZpbi1jaXRhZGVsLWdlbmVyYXRpdmUtYWktaHlwZS1vcGVuYWktbWlyYS1tdXJhdGktbnZpZGlhLWpvYnMv0gEA
2024-07-02 12:42:26,302 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs

...

2024-07-02 12:45:33,176 - AInewsbot - INFO - Processing https://www.cnet.com/tech/services-and-software/meta-is-changing-artificial-intelligence-labels-after-real-photos-were-marked-as-ai/
2024-07-02 12:45:33,177 - AInewsbot - INFO - get_url(https://www.cnet.com/tech/services-and-software/meta-is-changing-artificial-intelligence-labels-after-real-photos-were-marked-as-ai/) - starting get_files https://www.cnet.com/tech/services-and-software/meta-is-changing-artificial-intelligence-labels-after-real-photos-were-marked-as-ai/
2024-07-02 12:45:37,444 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/web

2024-07-02 12:45:48,058 - AInewsbot - INFO - get_url(Photographers__complaints_lead_Meta_to_revise_AI-generated_image_labels_on_Facebook__Instagram) - Saving Photographers__complaints_lead_Meta_to_revise_AI-generated_image_labels_on_Facebook__Instagram_20240702_124548.html as utf-8
2024-07-02 12:45:48,059 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiNGh0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8wMi9maWdtYS1haS13ZWF0aGVyLWFwcC_SAQA
2024-07-02 12:45:48,059 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiNGh0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8wMi9maWdtYS1haS13ZWF0aGVyLWFwcC_SAQA) - starting get_files https://news.google.com/articles/CBMiNGh0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8wMi9maWdtYS1haS13ZWF0aGVyLWFwcC_SAQA
2024-07-02 12:45:48,260 - AInewsbot - INFO - get_url(Meta_updates__Made_by_AI__label_across_Facebook__Instagram__Here_s_what_is_changing_-_Times_of_India) - Saving Meta_updates__Made_by_AI__label_across_Facebook__Instagram__Here_s_what_is_changing

2024-07-02 12:46:05,231 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnBjbWFnLmNvbS9uZXdzL3lvdXR1YmUtbm93LXN1cHBvcnRzLXJlbW92YWwtcmVxdWVzdHMtZm9yLWFpLXZpZGVvcy10aGF0LWxvb2stc291bmQtbGlrZdIBAA) - starting get_files https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnBjbWFnLmNvbS9uZXdzL3lvdXR1YmUtbm93LXN1cHBvcnRzLXJlbW92YWwtcmVxdWVzdHMtZm9yLWFpLXZpZGVvcy10aGF0LWxvb2stc291bmQtbGlrZdIBAA
2024-07-02 12:46:08,758 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-02 

2024-07-02 12:46:20,018 - AInewsbot - INFO - get_url(Generate_AI_stickers_of_yourself_on_WhatsApp_using__Image_Me__feature_powered_by_Meta_AI_-_NotebookCheck_net_News) - Saving Generate_AI_stickers_of_yourself_on_WhatsApp_using__Image_Me__feature_powered_by_Meta_AI_-_NotebookCheck_net_News_20240702_124620.html as utf-8
2024-07-02 12:46:20,020 - AInewsbot - INFO - Processing https://www.androidcentral.com/apps-software/is-geminis-expanded-context-window-as-useful-as-we-thought
2024-07-02 12:46:20,020 - AInewsbot - INFO - get_url(https://www.androidcentral.com/apps-software/is-geminis-expanded-context-window-as-useful-as-we-thought) - starting get_files https://www.androidcentral.com/apps-software/is-geminis-expanded-context-window-as-useful-as-we-thought
2024-07-02 12:46:24,419 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#n

2024-07-02 12:46:41,110 - AInewsbot - INFO - get_url(LWiAI_Podcast__172_-_Claude_and_Gemini_updates__Gemma_2__GPT-4_Critic) - Saving LWiAI_Podcast__172_-_Claude_and_Gemini_updates__Gemma_2__GPT-4_Critic_20240702_124641.html as utf-8
2024-07-02 12:46:41,112 - AInewsbot - INFO - Processing https://techcrunch.com/2024/07/02/news-outlets-are-accusing-perplexity-of-plagiarism-and-unethical-web-scraping/
2024-07-02 12:46:41,113 - AInewsbot - INFO - get_url(https://techcrunch.com/2024/07/02/news-outlets-are-accusing-perplexity-of-plagiarism-and-unethical-web-scraping/) - starting get_files https://techcrunch.com/2024/07/02/news-outlets-are-accusing-perplexity-of-plagiarism-and-unethical-web-scraping/
2024-07-02 12:46:41,912 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://rem

In [45]:
# flatten results
saved_pages = []
for r in results:
    saved_pages.extend(r)
saved_pages
len(saved_pages)

111

In [46]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'url', 'title', 'path']
pages_df

,id,url,title,path
0,508,https://news.google.com/articles/CBMiR2h0dHBzO...,Sentient closes $85M seed round for open-sourc...,htmldata/Sentient_closes__85M_seed_round_for_o...
1,187,https://www.benzinga.com/news/24/07/39594885/e...,Elon Musk Reveals The Staggering Amount Of Nvi...,htmldata/Elon_Musk_Reveals_The_Staggering_Amou...
2,392,https://news.google.com/articles/CBMieWh0dHBzO...,Northern Data considers AI unit US IPO at up t...,htmldata/Northern_Data_considers_AI_unit_US_IP...
3,384,https://news.google.com/articles/CBMiemh0dHBzO...,"Nvidia may be appealing, but one fund manager ...",htmldata/Nvidia_may_be_appealing__but_one_fund...
4,396,https://news.google.com/articles/CBMieGh0dHBzO...,Robinhood Purchases Pluto to Provide AI-Powere...,htmldata/Robinhood_Buys_Pluto_to_Provide_AI-Po...
...,...,...,...,...
106,247,https://gizmodo.com/youtube-ai-content-deepfak...,You Can Ask YouTube to Take Down AI-Generated ...,htmldata/You_Can_Ask_YouTube_to_Take_Down_AI_C...
107,625,https://news.google.com/articles/CBMiWWh0dHBzO...,"How to use Meta AI in WhatsApp, Facebook and I...",htmldata/How_to_use_Meta_AI_in_WhatsApp__Faceb...
108,695,https://github.com/pretzelai/pretzelai/blob/ma...,Show HN: Adding Mistral Codestral and GPT-4o t...,htmldata/pretzelai_README_md_at_main___pretzel...
109,208,https://www.the-sun.com/news/11788500/roman-sh...,"TWISTED PIC Texas man, 30, arrested after usin...",htmldata/Texas_man__30__arrested_after__using_...


In [47]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [48]:
SYSTEM_PROMPT = """You are a summarization assistant. 
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will output Markdown format.
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.
"""


In [49]:
async def fetch_openai2(session, payload, i):
    """
    Asynchronously fetches a response from the OpenAI URL using an aiohttp ClientSession.

    Parameters:
    - session (aiohttp.ClientSession): The aiohttp ClientSession object used for making HTTP requests.
    - payload (dict): The payload to be sent in the request body as JSON.
    - i (int): an id to return, to allow us map summary to original request

    Returns:
    - dict: The full JSON response from the OpenAI API.

    Raises:
    - aiohttp.ClientError: If there is an error during the HTTP request.

    Example usage:
    ```
    async with aiohttp.ClientSession() as session:
        response = await fetch_openai(session, payload)
        print(response)
    ```
    """
    async with session.post(API_URL, headers=headers, json=payload) as response:
        retval = await response.json()
        return (i, retval)
    

In [50]:
async def fetch_all2(page_df):
    tasks = []
    responses = []
    async with aiohttp.ClientSession() as session:

        for row in page_df.itertuples():

            # Read the HTML file
            try:
                with open(row.path, 'r', encoding='utf-8') as file:
                    html_content = file.read()
            except:
                print(f"Skipping {row.id} : {row.path}")
                continue

            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')

            # Filter out script and style elements
            for script_or_style in soup(['script', 'style']):
                script_or_style.extract()

            # Get text and strip leading/trailing whitespace
            visible_text = soup.get_text(separator=' ', strip=True)
            visible_text = visible_text[:MAX_INPUT_TOKENS]

            userprompt = f"""Summarize the main points of the following text concisely in at most 3 bullet points:
            {visible_text}
            """

            payload = {"model":  "gpt-3.5-turbo-0125",
                        "messages": [{"role": "system",
                                      "content": SYSTEM_PROMPT
                                     },
                                     {"role": "user",
                                      "content": userprompt
                                     }]
                       }

            task = asyncio.create_task(fetch_openai2(session, payload, row.id))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        return responses


In [51]:
print(datetime.now())
responses = await fetch_all2(AIdf)
print(datetime.now())
print(len(responses))
responses[0]


2024-07-02 12:46:59.394251
2024-07-02 12:47:06.813204
111


(438,
 {'id': 'chatcmpl-9gawhVgi5eIFkYjWrObNsNcqDL5y5',
  'object': 'chat.completion',
  'created': 1719938823,
  'model': 'gpt-3.5-turbo-0125',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "- AI-Crypto startup raised $85 million in deal led by Thiel's fund\n- Job openings increase surprises Wall Street\n- Powell encouraged by cooler inflation data"},
    'logprobs': None,
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 1914,
   'completion_tokens': 33,
   'total_tokens': 1947},
  'system_fingerprint': None})

In [52]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

111

In [53]:
from IPython.display import HTML, Image, Markdown, display
import markdown


In [63]:
markdown_str = ''

for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title}]({row.url})  \n\n{response_dict[row.id]} \n\n"
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. AI-Crypto Startup Raises $85 Million in Deal Led by Thiel's Fund](https://news.google.com/articles/CBMiSWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9haS1jcnlwdG8tc3RhcnR1cC1yYWlzZXMtODUtMTMwMDAwNTUzLmh0bWzSAQA)  

- AI-Crypto startup raised $85 million in deal led by Thiel's fund
- Job openings increase surprises Wall Street
- Powell encouraged by cooler inflation data 



[2. Sentient closes $85M seed round for open-source AI](https://news.google.com/articles/CBMiR2h0dHBzOi8vY29pbnRlbGVncmFwaC5jb20vbmV3cy9zZW50aWVudC04NS1taWxsaW9uLXJvdW5kLW9wZW4tc291cmNlLWFp0gEA)  

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain and AI technologies, with venture capital investments in blockchain startups increasing significantly in 2024. 



[3. Peter Thiel's Founders Fund Leads $85M Seed Investment Into Open-Source AI Platform Sentient](https://news.google.com/articles/CBMiiAFodHRwczovL3d3dy5jb2luZGVzay5jb20vYnVzaW5lc3MvMjAyNC8wNy8wMi9wZXRlci10aGllbHMtZm91bmRlcnMtZnVuZC1sZWFkcy04NW0tc2VlZC1pbnZlc3RtZW50LWludG8tb3Blbi1zb3VyY2UtYWktcGxhdGZvcm0tc2VudGllbnQv0gGMAWh0dHBzOi8vd3d3LmNvaW5kZXNrLmNvbS9idXNpbmVzcy8yMDI0LzA3LzAyL3BldGVyLXRoaWVscy1mb3VuZGVycy1mdW5kLWxlYWRzLTg1bS1zZWVkLWludmVzdG1lbnQtaW50by1vcGVuLXNvdXJjZS1haS1wbGF0Zm9ybS1zZW50aWVudC9hbXAv)  

- AI Platform Sentient raises $85M seed round 
- Various websites use cookies for personalization, analytics, and ads
- Different types of cookies are used for website functionality, caching, session management, and GDPR compliance 



[4. Peter Thiel's Founders Fund co-leads $85 million seed round for open-source AI platform Sentient](https://news.google.com/articles/CBMiWWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzAyOTgzL3BldGVyLXRoaWVsLWZvdW5kZXJzLWZ1bmQtc2VudGllbnQtZnVuZGluZy1haS1wb2x5Z29u0gFdaHR0cHM6Ly93d3cudGhlYmxvY2suY28vYW1wL3Bvc3QvMzAyOTgzL3BldGVyLXRoaWVsLWZvdW5kZXJzLWZ1bmQtc2VudGllbnQtZnVuZGluZy1haS1wb2x5Z29u)  

- Peter Thiel's Founders Fund co-leads $85 million seed round for the open-source AI platform Sentient with other notable investors.
- Sentient aims to build open-source AI models with community contributions, distinguishing itself from centralized AI models like OpenAI by allowing direct access and contributions to the model itself.
- Sentient plans to reward contributors with co-ownership of AI models and future rewards on usage, and will integrate with Polygon by building on the Polygon Chain Development Kit for its own chain launch. 



[5. Elon Musk says Grok 2 due in August, Grok 3 by end of year](https://news.google.com/articles/CBMiVmh0dHBzOi8vd3d3Lm5lb3dpbi5uZXQvbmV3cy9lbG9uLW11c2stc2F5cy1ncm9rLTItZHVlLWluLWF1Z3VzdC1ncm9rLTMtYnktZW5kLW9mLXllYXIv0gEA)  

- Elon Musk announced the release of Grok 2 in August and Grok 3 by the end of the year
- Grok 3 will be trained on 100,000 Nvidia H100s and is expected to be a significant advancement
- Grok is a language model developed by Elon Musk, with Grok 1.5 being the most recent version with improved reasoning capabilities 



...

[107. TWISTED PIC Texas man, 30, arrested after using deep fakeartificial intelligenceprogram to undress underage girl](https://www.the-sun.com/news/11788500/roman-shoffner-texas-ai-artificial-intelligence-teen-clothing-removed/)  

- Roman Shoffner, a 30-year-old man from Houston, was arrested for using artificial intelligence to undress an underage girl in photos.
- Shoffner altered photos using an AI program on his phone, removing the girl's clothing and making her appear naked.
- Police seized Shoffner's phone, leading to the discovery of the altered photos and his subsequent arrest on charges of possessing child pornography. 



[108. PowerWash Simulator adds ten Xbox achievements with Alice's Adventures DLC](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRydWVhY2hpZXZlbWVudHMuY29tL25ld3MvYWxpY2VzLWFkdmVudHVyZXMtc3BlY2lhbC1wYWNrLWRsYy1hY2hpZXZlbWVudHPSAQA)  

- PowerWash Simulator's Alice's Adventures DLC offers ten new Xbox achievements set in a fairy tale Wonderland with new levels, items, outfits, and tasks.
- The DLC includes achievements for completing different levels and specific actions within them, adding to the overall gaming experience.
- Players can unlock a total of 180 Gamerscore by completing tasks like cleaning, balancing on objects, and rolling croquet balls in specific locations within the game. 



[109. News outlets are accusing Perplexity of plagiarism and unethical web scraping](https://techcrunch.com/2024/07/02/news-outlets-are-accusing-perplexity-of-plagiarism-and-unethical-web-scraping/)  

- Perplexity AI is being accused of plagiarism and unethical web scraping by news outlets 
- Accusations include plagiarizing articles and illicitly scraping websites despite claims of compliance with Robots Exclusion Protocol 
- Perplexity's approach to using AI to generate answers from the internet is being scrutinized for potential copyright violations and unethical practices 



[110. Assassin's Creed 'fans' are attempting to get Shadows cancelled](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmdhbWluZ2JpYmxlLmNvbS9uZXdzL2Fzc2Fzc2lucy1jcmVlZC1zaGFkb3dzLWF0dGVtcHRpbmctY2FuY2VsbGVkLTE0NTU5Ny0yMDI0MDcwMtIBAA)  

- Subset of Assassin's Creed fans petitioning for cancellation of Assassin’s Creed Shadows due to concerns about cultural accuracy, specifically questioning the depiction of a black samurai.
- Petition calls for cancellation of the game citing lack of historical accuracy, disrespect to Japanese culture, and potential links to Asian racism, demanding Ubisoft to show sincere research and respect.
- Despite the backlash, Ubisoft is unlikely to cancel the release of Assassin’s Creed Shadows, set to launch on November 12 for Xbox Series X/S, PlayStation 5, and PC. 



[111. Call of Dutys New Extra Crispy Gun Lets You Cook, Eat, and Turn Your Enemies Into Fried Chicken and People Are Blaming It All on Fortnite Once Again](https://news.google.com/articles/CBMirwFodHRwczovL3d3dy5pZ24uY29tL2FydGljbGVzL2NhbGwtb2YtZHV0eXMtbmV3LWV4dHJhLWNyaXNweS1ndW4tbGV0cy15b3UtY29vay1lYXQtYW5kLXR1cm4teW91ci1lbmVtaWVzLWludG8tZnJpZWQtY2hpY2tlbi1hbmQtcGVvcGxlLWFyZS1ibGFtaW5nLWl0LWFsbC1vbi1mb3J0bml0ZS1vbmNlLWFnYWlu0gEA)  

- The introduction of the 'Extra Crispy' gun in Call of Duty Warzone Mobile has sparked debate and reactions from fans, with some finding it silly and others embracing the humor and creativity.
- Activision is known for incorporating outlandish cosmetic items and crossovers in Call of Duty games, generating significant revenue through microtransactions.
- With the release of Black Ops 6, there will be a fresh start for cosmetics, as bundles purchased for previous games will not carry over, signaling a return to a semblance of realism initially. 



In [56]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
display(HTML(html_str))


In [57]:
log("Sending mail")
from_addr = os.getenv("GMAIL_USER")
to_addr = os.getenv("GMAIL_USER")
subject = 'AI news summaries ' + datetime.now().strftime('%H:%M:%S')
body = f"""
<html>
    <head></head>
    <body>
    <div>
    {html_str}
    </div>
    </body>
</html>
"""

# Setup the MIME
message = MIMEMultipart()
message['From'] = os.getenv("GMAIL_USER")
message['To'] = os.getenv("GMAIL_USER")
message['Subject'] = subject
message.attach(MIMEText(body, 'html'))

# Create SMTP session
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()  # Secure the connection
    server.login(os.getenv("GMAIL_USER"), os.getenv("GMAIL_PASSWORD"))
    text = message.as_string()
    server.sendmail(from_addr, to_addr, text)

log("Finished")


2024-07-02 12:47:31,441 - AInewsbot - INFO - Sending mail
2024-07-02 12:47:34,623 - AInewsbot - INFO - Finished


20

In [64]:
print(markdown_str[:1000])


[1. AI-Crypto Startup Raises $85 Million in Deal Led by Thiel's Fund](https://news.google.com/articles/CBMiSWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9haS1jcnlwdG8tc3RhcnR1cC1yYWlzZXMtODUtMTMwMDAwNTUzLmh0bWzSAQA)  

- AI-Crypto startup raised $85 million in deal led by Thiel's fund
- Job openings increase surprises Wall Street
- Powell encouraged by cooler inflation data 

[2. Sentient closes $85M seed round for open-source AI](https://news.google.com/articles/CBMiR2h0dHBzOi8vY29pbnRlbGVncmFwaC5jb20vbmV3cy9zZW50aWVudC04NS1taWxsaW9uLXJvdW5kLW9wZW4tc291cmNlLWFp0gEA)  

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain 

In [94]:
BIGPROMPT = f"""You are a summarization assistant. I will provide a list of today's news stories about AI
and summary bullet points in markdown format. You are tasked with identifying and summaerizing the key themes, 
common facts, and recurring elements. Your goal is to create a concise summary containing about 20 of the most frequently 
mentioned topics and developments.

Example Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)  

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain 

Examples of important stories:

Major investments and funding rounds
Key technological advancements or breakthroughs
Frequently mentioned companies, organizations, or figures
Notable statements by AI leaders
Any other recurring themes or notable patterns

Instructions:
Read the summaries closely. 
Use only information provided in them.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Provide the most common facts without commentary 
Be concise and focus on the most common elements across the bullet points.
Group similar items together.
Ensure that you provide at least one link from the provided text for each item in the summary.
You must include at least 10 and no more than 25 items in the summary.

Bullet Points to Summarize:

{markdown_str}

"""

In [95]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "system", "content": SYSTEM_PROMPT},
              {"role": "user", "content": BIGPROMPT
              }],
    n=5,   
    temperature=0.5
)


2024-07-02 13:45:52,351 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [96]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


- Sentient raised \$85 million in seed funding led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform, focusing on incentivizing AI development through blockchain. [source](https://news.google.com/articles/CBMiR2h0dHBzOi8vY29pbnRlbGVncmFwaC5jb20vbmV3cy9zZW50aWVudC04NS1taWxsaW9uLXJvdW5kLW9wZW4tc291cmNlLWFp0gEA)
- Elon Musk announced the upcoming releases of Grok 2 in August and Grok 3 by the end of the year, planning to train Grok 3 on 100,000 Nvidia H100 GPUs. His AI startup, xAI, aims to push boundaries in AI technology. [source](https://news.google.com/articles/CBMiXGh0dHBzOi8vY29pbmdhcGUuY29tL2Vsb24tbXVzay1hbm5vdW5jZXMtamFydmlzLWluc3BpcmVkLXhhaS1ncm9rLTItYWktY2hhdGJvdC1yZWxlYXNlLWRhdGUv0gFgaHR0cHM6Ly9jb2luZ2FwZS5jb20vZWxvbi1tdXNrLWFubm91bmNlcy1qYXJ2aXMtaW5zcGlyZWQteGFpLWdyb2stMi1haS1jaGF0Ym90LXJlbGVhc2UtZGF0ZS9hbXAv)
- Northern Data AG is considering a U.S. IPO for its AI cloud computing and data center businesses at a valuation up to \$16 billion, reflecting a shift from Bitcoin mining to AI-focused operations. [source](https://news.google.com/articles/CBMieWh0dHBzOi8vd3d3LmNoYW5uZWxuZXdzYXNpYS5jb20vYnVzaW5lc3Mvbm9ydGhlcm4tZGF0YS1jb25zaWRlcnMtYWktdW5pdC11cy1pcG8tMTYtYmlsbGlvbi1ibG9vbWJlcmctbmV3cy1yZXBvcnRzLTQ0NTAwNjbSAQA)
- Ken Griffin expressed skepticism about AI's potential to replace jobs soon, emphasizing its role in enhancing human productivity rather than replacing workers entirely. [source](https://news.google.com/articles/CBMiZWh0dHBzOi8vZm9ydHVuZS5jb20vMjAyNC8wNy8wMi9rZW4tZ3JpZmZpbi1jaXRhZGVsLWdlbmVyYXRpdmUtYWktaHlwZS1vcGVuYWktbWlyYS1tdXJhdGktbnZpZGlhLWpvYnMv0gEA)
- Nvidia CEO Jensen Huang envisions a future where advanced AI systems outperform humans in the job market, driving transformative changes across various industries. [source](https://www.windowscentral.com/software-apps/nvidias-ceo-envisions-a-future-where-multi-shot-agentic-ais-armed-with-reasoning-superpowers-outperform-humans-in-the-job-market)
- SAP's strategy emphasizes leveraging AI for business transformation, cloud migration, and sustainability, with partnerships with tech giants for improved functionalities and processes. [source](https://news.google.com/articles/CBMid2h0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvbW9vcmluc2lnaHRzLzIwMjQvMDYvMjcvc2FwLWNoYXJ0cy1hLWNvdXJzZS1mb3ItdGhlLWFpLWRyaXZlbi1lbnRlcnByaXNlLWF0LXNhcHBoaXJlLTIwMjQv0gEA)
- China aims to set at least 50 AI standards by 2026, covering aspects such as chips, applications, and safety, aligning with international AI standards for inclusive business environments. [source](https://www.scmp.com/tech/tech-trends/article/3268907/china-eyes-least-50-sets-ai-standards-2026-chips-applications-and-safety)
- AI-generated exam papers are detected in only 6% of cases, raising concerns about academic integrity and the need for updated policies in the educational sector. [source](https://news.google.com/articles/CBMiamh0dHBzOi8vd3d3LmV1cm9uZXdzLmNvbS9uZXh0LzIwMjQvMDcvMDIvYWktZ2VuZXJhdGVkLWV4YW0tcGFwZXJzLWdvLXVuZGV0ZWN0ZWQtaW4tOTQtb2YtY2FzZXMtc3R1ZHktZmluZHPSAQA)
- California's proposed AI regulation bill, opposed by tech giants like Meta, seeks to prevent catastrophic events caused by AI models by implementing safety measures and oversight. [source](https://www.fastcompany.com/91150396/meta-is-against-californias-ai-bill-heres-why)
- AI companies face growing pressure to pay for training data, impacting the AI industry by potentially leading to the development of smaller, more efficient models. [source](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LnRlY2hub2xvZ3lyZXZpZXcuY29tLzIwMjQvMDcvMDIvMTA5NDUwOC9haS1jb21wYW5pZXMtYXJlLWZpbmFsbHktYmVpbmctZm9yY2VkLXRvLWNvdWdoLXVwLWZvci10cmFpbmluZy1kYXRhL9IBfGh0dHBzOi8vd3d3LnRlY2hub2xvZ3lyZXZpZXcuY29tLzIwMjQvMDcvMDIvMTA5NDUwOC9haS1jb21wYW5pZXMtYXJlLWZpbmFsbHktYmVpbmctZm9yY2VkLXRvLWNvdWdoLXVwLWZvci10cmFpbmluZy1kYXRhL2FtcC8)
- ElevenLabs partners with the estates of famous Hollywood actors to use AI-generated voices to read text on a new app, showcasing the merging of AI and entertainment. [source](https://news.google.com/articles/CBMibGh0dHBzOi8vdmFyaWV0eS5jb20vMjAyNC9iaXovbmV3cy9haS1qdWR5LWdhcmxhbmQtamFtZXMtZGVhbi1idXJ0LXJleW5vbGRzLWVsZXZlbmxhYnMtcmVhZGVyLWFwcC0xMjM2MDU4ODI5L9IBcGh0dHBzOi8vdmFyaWV0eS5jb20vMjAyNC9iaXovbmV3cy9haS1qdWR5LWdhcmxhbmQtamFtZXMtZGVhbi1idXJ0LXJleW5vbGRzLWVsZXZlbmxhYnMtcmVhZGVyLWFwcC0xMjM2MDU4ODI5L2FtcC8)
- Geoffrey Rush discussed the intrigue of human creativity versus AI sophistication, reflecting on his experiences and future projects at the Karlovy Vary International Film Festival. [source](https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmhvbGx5d29vZHJlcG9ydGVyLmNvbS9tb3ZpZXMvbW92aWUtbmV3cy9nZW9mZnJleS1ydXNoLWFpLWpvaG4tbGl0aGdvdy1ncm91Y2hvLW1hcngtZmlsbS0xMjM1OTM3MjAxL9IBc2h0dHBzOi8vd3d3LmhvbGx5d29vZHJlcG9ydGVyLmNvbS9tb3ZpZXMvbW92aWUtbmV3cy9nZW9mZnJleS1ydXNoLWFpLWpvaG4tbGl0aGdvdy1ncm91Y2hvLW1hcngtZmlsbS0xMjM1OTM3MjAxL2FtcC8)
- Google faces rising emissions due to increased AI usage, impacting its climate change goals as data centers consume significant energy. [source](https://www.ft.com/content/383719aa-df38-4ae3-ab0e-6279a897915e)
- Crusoe Energy shifts focus from crypto mining to building AI data centers, leveraging its expertise in high-density GPU hosting and expanding data center capacity. [source](https://news.google.com/articles/CBMiZ2h0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9jcnVzb2UtcGl2b3RzLXRvLWJ1aWxkaW5nLWRhdGEtY2VudGVycy1wbGFucy0xMDBtdy1mYWNpbGl0eS_SAQA)
- Anthropic launches a program to fund reliable AI benchmarks, incentivizing researchers to develop new evaluation methods to enhance AI's societal impact assessment. [source](https://siliconangle.com/2024/07/01/anthropic-launches-new-program-fund-creation-reliable-ai-benchmarks/)
- Nvidia is facing its first AI antitrust charge due to its dominant market position, prompting regulatory scrutiny from France, the EU, and the US. [source](https://news.google.com/articles/CBMiPWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL252aWRpYS1hbnRpdHJ1c3QtYWktNzhjMWM1NDjSAUFodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvbnZpZGlhLWFudGl0cnVzdC1haS03OGMxYzU0OA)
- Meta revises "Made with AI" labels to "AI Info" on Facebook and Instagram posts for added transparency and to prevent confusion about AI-generated content. [source](https://news.google.com/articles/CBMiVmh0dHBzOi8vd3d3LnNvY2lhbG1lZGlhdG9kYXkuY29tL25ld3MvbWV0YS1zd2l0Y2hlcy1haS1sYWJlbHMtbWFkZS1haS1pbmZvLXRhZy83MjAzODQv0gEA)
- AI tools used in creating exploitative content are drawing legal and regulatory responses as lawmakers seek to address abuse and hold platforms accountable. [source](https://www.star-telegram.com/news/state/texas/article289691769.html)
- WhatsApp introduces "Imagine Me" feature powered by Meta AI, allowing users to generate personalized images enhancing the user experience on the platform. [source](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy5ub3RlYm9va2NoZWNrLm5ldC9HZW5lcmF0ZS1BSS1zdGlja2Vycy1vZi15b3Vyc2VsZi1vbi1XaGF0c0FwcC11c2luZy1JbWFnZS1NZS1mZWF0dXJlLXBvd2VyZWQtYnktTWV0YS1BSS44NTU5NTkuMC5odG1s0gEA)
- AI use in supply chains and logistics is set to enhance efficiency and decision-making, transforming industry processes and operations. [source](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1hZXJzay5jb20vaW5zaWdodHMvZGlnaXRhbGlzYXRpb24vMjAyNC8wNy8wMi9haS1pbi1sb2dpc3RpY3MtYW5kLXN1cHBseS1jaGFpbnPSAQA)

In [89]:
display(Markdown(response_str[11:]))



- **Major Investments and Funding Rounds**: AI-Crypto startup Sentient raised \$85M in seed funding led by Peter Thiel's Founders Fund, consistently reported across multiple sources ([Sentient closes \$85M seed round for open-source AI](https://news.google.com/articles/CBMiSWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9haS1jcnlwdG8tc3RhcnR1cC1yYWlzZXMtODUtMTMwMDAwNTUzLmh0bWzSAQA), [Sentient closes \$85M seed round for open-source AI](https://news.google.com/articles/CBMiR2h0dHBzOi8vY29pbnRlbGVncmFwaC5jb20vbmV3cy9zZW50aWVudC04NS1taWxsaW9uLXJvdW5kLW9wZW4tc291cmNlLWFp0gEA) and [Peter Thiel’s Founders Fund co-leads \$85 million seed round for open-source AI platform Sentient](https://news.google.com/articles/CBMiWWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzAyOTgzL3BldGVyLXRoaWVsLWZvdW5kZXJzLWZ1bmQtc2VudGllbnQtZnVuZGluZy1haS1wb2x5Z29u0gFdaHR0cHM6Ly93d3cudGhlYmxvY2suY28vYW1wL3Bvc3QvMzAyOTgzL3BldGVyLXRoaWVsLWZvdW5kZXJzLWZ1bmQtc2VudGllbnQtZnVuZGluZy1haS1wb2x5Z29u)).

- **Elon Musk’s Grok Chatbots**: Elon Musk's xAI announced the launch schedule for Grok 2 and Grok 3, along with plans to train them on 100,000 Nvidia H100 GPUs, highlighting a significant investment in advanced AI hardware ([Elon Musk says Grok 2 due in August, Grok 3 by end of year](https://news.google.com/articles/CBMiVmh0dHBzOi8vd3d3Lm5lb3dpbi5uZXQvbmV3cy9lbG9uLW11c2stc2F5cy1ncm9rLTItZHVlLWluLWF1Z3VzdC1ncm9rLTMtYnktZW5kLW9mLXllYXIv0gEA), [Elon Musk Announces JARVIS-Inspired xAI Grok 2 AI Chatbot Release Date](https://news.google.com/articles/CBMiXGh0dHBzOi8vY29pbmdhcGUuY29tL2Vsb24tbXVzay1hbm5vdW5jZXMtamFydmlzLWluc3BpcmVkLXhhaS1ncm9rLTItYWktY2hhdGJvdC1yZWxlYXNlLWRhdGUv0gFgaHR0cHM6Ly9jb2luZ2FwZS5jb20vZWxvbi1tdXNrLWFubm91bmNlcy1qYXJ2aXMtaW5zcGlyZWQteGFpLWdyb2stMi1haS1jaGF0Ym90LXJlbGVhc2UtZGF0ZS9hbXAv) and [Elon Musk Reveals The Staggering Amount Of Nvidia H100 GPUs That Will Be Used To Train xAI Chatbot Grok](https://www.benzinga.com/news/24/07/39594885/elon-musk-reveals-the-staggering-amount-of-nvidia-h100-gpus-that-will-be-used-to-train-xai-chatbot-g)).

- **Regulatory and Ethical Concerns**: Regulatory bodies like French and European authorities, as well as the US Department of Justice, are investigating Nvidia for potential antitrust issues due to its dominance in the AI industry, reflecting growing scrutiny of AI monopolies ([French regulators consider AI antitrust charges against Nvidia](https://news.google.com/articles/CBMiQWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS8yMDI0LzA3LzAxL2ZyZW5jaF9udmlkaWFfY29tcGV0aXRpb24v0gFFaHR0cHM6Ly93d3cudGhlcmVnaXN0ZXIuY29tL0FNUC8yMDI0LzA3LzAxL2ZyZW5jaF9udmlkaWFfY29tcGV0aXRpb24v), [Nvidia's dominance in AI market under scrutiny](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vbnZpZGlhLXNlY3JldC1zYXVjZS1yZWd1bGF0b3JzLWdwdS1jaGlwcy1qZW5zZW4taHVhbmctMjAyNC030gFgaHR0cHM6Ly93d3cuYnVzaW5lc3NpbnNpZGVyLmNvbS9udmlkaWEtc2VjcmV0LXNhdWNlLXJlZ3VsYXRvcnMtZ3B1LWNoaXBzLWplbnNlbi1odWFuZy0yMDI0LTc_YW1w) and [Nvidia faces first AI antitrust charge](https://news.google.com/articles/CBMiPWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL252aWRpYS1hbnRpdHJ1c3QtYWktNzhjMWM1NDjSAUFodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvbnZpZGlhLWFudGl0cnVzdC1haS03OGMxYzU0OA)). 
